# Prerequisites
Please run the prerequiste cells before trying the examples.

## Install python module dependencies

In [ ]:
import sys
def installModule(projectName:str, moduleName:str=None):
    '''Installs and loads the given module if not already installed'''
    if moduleName is None:
        moduleName=projectName
    !python -m pip install -U --no-input $projectName
    print(f'{projectName} installed')
    %reload_ext $moduleName

installModule('jupyter-xml')
installModule('jupyter-rdfify==1.0.1','jupyter-rdfify')
installModule('SPARQLWrapper')
installModule('tabulate')
installModule("pylodstorage", "lodstorage")

# Querying wikidata
[wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page) provides an [SPARQL endpoint](https://query.wikidata.org/) for querying their triplestore. 

To get a response from the endpoint the query is sent as post request and returns the results in __json__ or __xml__ format.

The result is then post-processed into a more usable format. Depending on the post-processing the results are converted into python types and this step can also include a merging of results. For example if a proeprty has multiple values and all match the query each value will result in its own row (For more details in the result reduction see [SPARQL Transformer](https://www.eurecom.fr/en/publication/5927/download/data-publi-5927.pdf)).

[![](https://mermaid.ink/img/eyJjb2RlIjoiZmxvd2NoYXJ0IExSXG4gICAgQShRdWVyeSkgLS0-fHBvc3QgcmVxdWVzdHwgQihldmFsdWF0ZSBRdWVyeSlcbiAgICBCIC0tPnxyZXNwb25zZXwgQyhKU09OIFJlc3BvbnNlKVxuICAgIEMgLS0-fHBvc3QgcHJvY2Vzc3wgRChcIkxpc3Qgb2YgRGljdHMgKExvRClcIikiLCJtZXJtYWlkIjp7InRoZW1lIjoiZGVmYXVsdCJ9LCJ1cGRhdGVFZGl0b3IiOmZhbHNlLCJhdXRvU3luYyI6dHJ1ZSwidXBkYXRlRGlhZ3JhbSI6ZmFsc2V9)](https://mermaid.live/edit/#eyJjb2RlIjoiZmxvd2NoYXJ0IExSXG4gICAgQShRdWVyeSkgLS0-fHBvc3QgcmVxdWVzdHwgQihldmFsdWF0ZSBRdWVyeSlcbiAgICBCIC0tPnxyZXNwb25zZXwgQyhKU09OIFJlc3BvbnNlKVxuICAgIEMgLS0-fHBvc3QgcHJvY2Vzc3wgRChcIkxpc3Qgb2YgRGljdHMgKExvRClcIikiLCJtZXJtYWlkIjoie1xuICBcInRoZW1lXCI6IFwiZGVmYXVsdFwiXG59IiwidXBkYXRlRWRpdG9yIjpmYWxzZSwiYXV0b1N5bmMiOnRydWUsInVwZGF0ZURpYWdyYW0iOmZhbHNlfQ)


[![](https://mermaid.ink/img/eyJjb2RlIjoic2VxdWVuY2VEaWFncmFtIFxuICAgIHBhcnRpY2lwYW50IEEgYXMgQ2xpZW50XG4gICAgcGFydGljaXBhbnQgQiBhcyBTUEFSUUwgRW5kcG9pbnRcbiAgICBwYXJ0aWNpcGFudCBDIGFzIFRyaXBsZXN0b3JlXG4gICAgYWN0aXZhdGUgQVxuICAgIEEtPj5COiBwb3N0IFF1ZXJ5XG4gICAgYWN0aXZhdGUgQlxuICAgIGxvb3AgUXVlcnkgRXZhbHVhdGlvblxuICAgICAgICBhY3RpdmF0ZSBDXG4gICAgICAgIEItLT4-QzogXG4gICAgICAgIG5vdGUgcmlnaHQgb2YgQzogRmluZCBtYXRjaGluZyB0cmlwbGVzXG4gICAgICAgIEMtLT4-QjogXG4gICAgICAgIGRlYWN0aXZhdGUgQ1xuICAgIGVuZFxuICAgIEItLT4-QTogSlNPTi9YTUwgUmVzcG9uc2VcbiAgICBkZWFjdGl2YXRlIEJcbiAgICBBLS0-PkE6IHBvc3QtcHJvY2VzcyByZXN1bHRzXG4gICAgZGVhY3RpdmF0ZSBBIiwibWVybWFpZCI6eyJ0aGVtZSI6ImRlZmF1bHQifSwidXBkYXRlRWRpdG9yIjpmYWxzZSwiYXV0b1N5bmMiOnRydWUsInVwZGF0ZURpYWdyYW0iOmZhbHNlfQ)](https://mermaid.live/edit/#eyJjb2RlIjoic2VxdWVuY2VEaWFncmFtIFxuICAgIHBhcnRpY2lwYW50IEEgYXMgQ2xpZW50XG4gICAgcGFydGljaXBhbnQgQiBhcyBTUEFSUUwgRW5kcG9pbnRcbiAgICBwYXJ0aWNpcGFudCBDIGFzIFRyaXBsZXN0b3JlXG4gICAgYWN0aXZhdGUgQVxuICAgIEEtPj5COiBwb3N0IFF1ZXJ5XG4gICAgYWN0aXZhdGUgQlxuICAgIGxvb3AgUXVlcnkgRXZhbHVhdGlvblxuICAgICAgICBhY3RpdmF0ZSBDXG4gICAgICAgIEItLT4-QzogXG4gICAgICAgIG5vdGUgcmlnaHQgb2YgQzogRmluZCBtYXRjaGluZyB0cmlwbGVzXG4gICAgICAgIEMtLT4-QjogXG4gICAgICAgIGRlYWN0aXZhdGUgQ1xuICAgIGVuZFxuICAgIEItLT4-QTogSlNPTi9YTUwgUmVzcG9uc2VcbiAgICBkZWFjdGl2YXRlIEJcbiAgICBBLS0-PkE6IHBvc3QtcHJvY2VzcyByZXN1bHRzXG4gICAgZGVhY3RpdmF0ZSBBIiwibWVybWFpZCI6IntcbiAgXCJ0aGVtZVwiOiBcImRlZmF1bHRcIlxufSIsInVwZGF0ZUVkaXRvciI6ZmFsc2UsImF1dG9TeW5jIjp0cnVlLCJ1cGRhdGVEaWFncmFtIjpmYWxzZX0)

## Example
Query all __James Bond__ movies
### Sending the Query as posts request

In [2]:
import requests

url = "https://query.wikidata.org/sparql"
query="""#James Bond Movies
SELECT ?item ?movieTitle (MIN(?date) AS ?firstReleased)
WHERE {
  ?item wdt:P179 wd:Q2484680;
        rdfs:label ?movieTitle;
        wdt:P577 ?date.
  FILTER(lang(?movieTitle)="en")
} GROUP BY ?item ?movieTitle
ORDER BY DESC(?firstReleased)
LIMIT 3
"""
params={
    "query":query,
    "format":"xml" # or json
}
response = requests.request("POST", url, params=params)

print(response.text)

<?xml version='1.0' encoding='UTF-8'?>
<sparql xmlns='http://www.w3.org/2005/sparql-results#'>
	<head>
		<variable name='item'/>
		<variable name='movieTitle'/>
		<variable name='firstReleased'/>
	</head>
	<results>
		<result>
			<binding name='item'>
				<uri>http://www.wikidata.org/entity/Q21534241</uri>
			</binding>
			<binding name='movieTitle'>
				<literal xml:lang='en'>No Time to Die</literal>
			</binding>
			<binding name='firstReleased'>
				<literal datatype='http://www.w3.org/2001/XMLSchema#dateTime'>2021-09-28T00:00:00Z</literal>
			</binding>
		</result>
		<result>
			<binding name='item'>
				<uri>http://www.wikidata.org/entity/Q18602670</uri>
			</binding>
			<binding name='movieTitle'>
				<literal xml:lang='en'>Spectre</literal>
			</binding>
			<binding name='firstReleased'>
				<literal datatype='http://www.w3.org/2001/XMLSchema#dateTime'>2015-10-26T00:00:00Z</literal>
			</binding>
		</result>
		<result>
			<binding name='item'>
				<uri>http://www.wikidata.org/en

### Post-Process the Response

In [3]:
from xml.etree import ElementTree
from tabulate import tabulate
tree=ElementTree.fromstring(response.text)
prefix_map={"res":"http://www.w3.org/2005/sparql-results#"}
lod=[]
for result in tree.findall(".//res:results/res:result", prefix_map):
    record={}
    for binding in result.findall("./res:binding", prefix_map):
        name=binding.attrib['name']
        value=binding.find("./*").text
        record[name]=value
    lod.append(record)

# pretty print the list of dicts (LoD)
print(tabulate(lod, headers="keys"))

item                                      movieTitle      firstReleased
----------------------------------------  --------------  --------------------
http://www.wikidata.org/entity/Q21534241  No Time to Die  2021-09-28T00:00:00Z
http://www.wikidata.org/entity/Q18602670  Spectre         2015-10-26T00:00:00Z
http://www.wikidata.org/entity/Q4941      Skyfall         2012-10-23T00:00:00Z


# SPARQL Libraries
To simplify the querying process there existdifferent python libraries that take care of the post-processing of the result and directly return the result in  python types. Such Libraries are:

|Library| Description|
|---|---|
|RDFify| SPARQL support for jupyter notebooks with the use of jupyter magicwords (based on SPARQLWrapper)|
|SPARQLWrapper| |
|pyLODStorage| supports vonversion of the results into other storage formats and provides a documentation of the executed query (based on SPARQLWrapper)|

In the following the mentioned libraries are used to execute and printout the result of the following query:

```sparql
SELECT ?substance ?substanceLabel ?formula ?structure ?CAS
WHERE { 
  ?substance wdt:P31 wd:Q11173.
  ?substance wdt:P231 ?CAS.
  ?substance wdt:P274 ?formula.
  ?substance wdt:P117  ?structure.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 15
```
> see [pyLodStorage Random Substances with CAS number example](http://wiki.bitplan.com/index.php/PyLoDStorage#15_Random_substances_with_CAS_number)

## RDFify
### RDFify tool documentation

In [4]:
%rdf turtle -h

usage: %rdf turtle [-h] [--serialize {turtle,json-ld,xml,n3}]
                   [--display {graph,table,raw,none}] [--label LABEL]
                   [--prefix] [--entail {rdfs,owl,rdfs+owl}]

options:
  -h, --help            show this help message and exit
  --serialize {turtle,json-ld,xml,n3}, -s {turtle,json-ld,xml,n3}
                        Format for serializing when display is set to raw.
  --display {graph,table,raw,none}, -d {graph,table,raw,none}
                        How output is displayed
  --label LABEL, -l LABEL
                        Store graph locally with this label
  --prefix, -p          Define a prefix which gets prepend to every query.
                        Useful for PREFIX declarations
  --entail {rdfs,owl,rdfs+owl}, -e {rdfs,owl,rdfs+owl}
                        Uses a brute force implementation of the finite
                        version of RDFS semantics or OWL 2 RL. Uses owlrl
                        python package.



In [5]:
%rdf sparql -h

usage: %rdf sparql [-h] [--endpoint ENDPOINT] [--format {xml,json}]
                   [--display {graph,table,raw,none}]
                   [--prefix | --local LOCAL] [--store STORE]

options:
  -h, --help            show this help message and exit
  --endpoint ENDPOINT, -e ENDPOINT
                        SPARQL endpoint
  --format {xml,json}, -f {xml,json}
                        Requested format for query result
  --display {graph,table,raw,none}, -d {graph,table,raw,none}
                        How output is displayed. Does not work for local
                        queries.
  --prefix, -p          Define a prefix which gets prepend to every query.
                        Useful for PREFIX declarations
  --local LOCAL, -l LOCAL
                        Give a label of a local graph. This cell will then
                        ignore the endpoint and query the graph instead
  --store STORE, -s STORE
                        Store result of the query with this label



In [6]:
%rdf --help

usage: %rdf [-h] [--verbose] [--return-store]
            {turtle,n3,json-ld,xml,sparql,shex,graph} ...

positional arguments:
  {turtle,n3,json-ld,xml,sparql,shex,graph}
                        RDF modules
    turtle              Turtle module
    n3                  Notation 3 module
    json-ld             JSON-LD module
    xml                 XML+RDF module
    sparql              SPARQL module
    shex                ShEx module
    graph               Graph management module

options:
  -h, --help            show this help message and exit
  --verbose, -v         Enable verbose output
  --return-store, -r    Returns a copy of all present elements (graphs,
                        schemas, etc.)



### RDFify Example Usage

In [7]:
%%rdf sparql --endpoint https://query.wikidata.org/sparql -d table -f json -s substances
SELECT ?substance ?substanceLabel ?formula ?structure ?CAS
WHERE { 
  ?substance wdt:P31 wd:Q11173.
  ?substance wdt:P231 ?CAS.
  ?substance wdt:P274 ?formula.
  ?substance wdt:P117  ?structure.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 15


substance,substanceLabel,formula,structure,CAS
<http://www.wikidata.org/entity/Q32703>,aminomethylpropanol@en,C₄H₁₁NO,<http://commons.wikimedia.org/wiki/Special:FilePath/2-amino-2-methyl-1-propanol.svg>,124-68-5
<http://www.wikidata.org/entity/Q32703>,aminomethylpropanol@en,C₄H₁₁NO,<http://commons.wikimedia.org/wiki/Special:FilePath/Isobutanolamine%20t.png>,124-68-5
<http://www.wikidata.org/entity/Q43656>,cholesterol@en,C₂₇H₄₆O,<http://commons.wikimedia.org/wiki/Special:FilePath/Structural%20formula%20of%20cholesterol.svg>,57-88-5
<http://www.wikidata.org/entity/Q44804>,RS-127445@en,C₁₇H₁₆FN₃,<http://commons.wikimedia.org/wiki/Special:FilePath/RS-127%2C445%20structure.png>,199864-87-4
<http://www.wikidata.org/entity/Q45143>,fulminic acid@en,CHNO,<http://commons.wikimedia.org/wiki/Special:FilePath/Fulminezuur.png>,506-85-4
<http://www.wikidata.org/entity/Q49546>,acetone@en,C₃H₆O,<http://commons.wikimedia.org/wiki/Special:FilePath/Acetone-2D-skeletal.svg>,67-64-1
<http://www.wikidata.org/entity/Q52858>,ethane@en,C₂H₆,<http://commons.wikimedia.org/wiki/Special:FilePath/Ethan%20Keilstrich.svg>,74-84-0
<http://www.wikidata.org/entity/Q58356>,amoxapine@en,C₁₇H₁₆ClN₃O,<http://commons.wikimedia.org/wiki/Special:FilePath/Amoxapine.svg>,14028-44-5
<http://www.wikidata.org/entity/Q58713>,clomipramine@en,C₁₉H₂₃ClN₂,<http://commons.wikimedia.org/wiki/Special:FilePath/Clomipramine.svg>,303-49-1
<http://www.wikidata.org/entity/Q68484>,prucalopride@en,C₁₈H₂₆ClN₃O₃,<http://commons.wikimedia.org/wiki/Special:FilePath/Prucalopride.svg>,179474-81-8


In [8]:
store = %rdf -r
g = store["rdfresults"]
print(g['substances']) # SPARQLWrapper QueryResult

{"requestedFormat" : 'json',
"response (a file-like object, as return by the urllib2.urlopen library call)" : {
	"url" : "https://query.wikidata.org/sparql?query=SELECT+%3Fsubstance+%3FsubstanceLabel+%3Fformula+%3Fstructure+%3FCAS%0AWHERE+%7B+%0A++%3Fsubstance+wdt%3AP31+wd%3AQ11173.%0A++%3Fsubstance+wdt%3AP231+%3FCAS.%0A++%3Fsubstance+wdt%3AP274+%3Fformula.%0A++%3Fsubstance+wdt%3AP117++%3Fstructure.%0A++SERVICE+wikibase%3Alabel+%7B+bd%3AserviceParam+wikibase%3Alanguage+%22%5BAUTO_LANGUAGE%5D%2Cen%22.+%7D%0A%7D%0ALIMIT+15%0A%0A&format=json&output=json&results=json",
	"code" : "200",
	"headers" : server: nginx/1.14.2
date: Wed, 19 Oct 2022 21:59:50 GMT
content-type: application/sparql-results+json;charset=utf-8
x-first-solution-millis: 4
x-served-by: wdqs1014
access-control-allow-origin: *
cache-control: public, max-age=300
vary: Accept, Accept-Encoding
age: 1
x-cache: cp3060 miss, cp3056 pass
x-cache-status: pass
server-timing: cache;desc="pass", host;desc="cp3056"
strict-transport-secu

## [SPARQLWrapper](https://sparqlwrapper.readthedocs.io/en/latest/main.html)
SPARQLWrapper result Format:
* head
  * vars (list of colimn names)
* results
  * bindings (list with all row values)
    * <column name>
       * type
       * value
    
An SPARWLWrapper result can be converted to an list of dicts (LoD) with the following list comprehentions:
```python
lod=[{column:cell["value"] for column, cell in row.items()} for row in results["results"]["bindings"]] 
```

In [9]:
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
from tabulate import tabulate
from IPython.display import Image, SVG
# List of Dicts (LoD) 

q = """
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wd: <http://www.wikidata.org/entity/>
SELECT ?substance ?substanceLabel ?formula ?structure ?CAS
WHERE { 
  ?substance wdt:P31 wd:Q11173.
  ?substance wdt:P231 ?CAS.
  ?substance wdt:P274 ?formula.
  ?substance wdt:P117  ?structure.
}
LIMIT 15
"""

sparql = SPARQLWrapper("https://qlever.cs.uni-freiburg.de/api/wikidata")
sparql.setQuery(q)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print(results)
# print raw results
#print(results)
lod=[{column:cell["value"] for column, cell in row.items()} for row in results["results"]["bindings"]]
print(tabulate(lod, headers="keys"))


{'head': {'vars': ['substance', 'substanceLabel', 'formula', 'structure', 'CAS']}, 'results': {'bindings': [{'CAS': {'type': 'literal', 'value': '102989-24-2'}, 'formula': {'type': 'literal', 'value': 'C₃₇H₆₆O₇'}, 'structure': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Asimicin.svg'}, 'substance': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q100138042'}}, {'CAS': {'type': 'literal', 'value': '373-53-5'}, 'formula': {'type': 'literal', 'value': 'CH₂FI'}, 'structure': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Fluoroiodomethane%20Formula%20V1.svg'}, 'substance': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1002152'}}, {'CAS': {'type': 'literal', 'value': '90566-53-3'}, 'formula': {'type': 'literal', 'value': 'C₂₂H₂₇F₃O₄S'}, 'structure': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Fluticasone.svg'}, 'substance': {'type': 'uri', 'value': 'http://www.wikidata.org/en

## pyLODStorage

pyLODStorage directly converts the query result into an list of dicts (LoD) and also provides methods to store the LoD in different formats such as JSON, SQL and CSV.

An additional advantage is that it provides a pretty print of the query and result in the jupyter environment. 

In [10]:
from IPython.display import display, Markdown
from lodstorage.query import QueryManager, Query
from lodstorage.sparql import SPARQL
import copy

endpoint=SPARQL("https://query.wikidata.org/sparql")

queryStr="""# List of 15 random chemical components with CAS-Number, formula and structure
# see also https://github.com/WolfgangFahl/pyLoDStorage/issues/46
# WF 2021-08-23
SELECT ?substance ?substanceLabel ?formula ?structure ?CAS
WHERE { 
?substance wdt:P31 wd:Q11173.
?substance wdt:P231 ?CAS.
?substance wdt:P274 ?formula.
?substance wdt:P117  ?structure.
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 15
"""
query=Query(query=queryStr, name="CAS15", title="15 Random substances with CAS number", lang="sparql",
            description="Wikidata SPARQL query showing the 15 random chemical substances with their CAS Number")
try:
    qlod=endpoint.queryAsListOfDicts(query.query)
    lod=copy.deepcopy(qlod)
    
    # pretty print the query and its result
    doc=query.documentQueryResult(lod, tablefmt="github",floatfmt=".0f",tryItUrl="https://query.wikidata.org")
    display(Markdown(str(doc)))
except Exception as ex:
            print(f"{query.title} at {endpoint.sparql.endpoint} failed: {ex}")

## 15 Random substances with CAS number
Wikidata SPARQL query showing the 15 random chemical substances with their CAS Number
### query
```sparql
# List of 15 random chemical components with CAS-Number, formula and structure
# see also https://github.com/WolfgangFahl/pyLoDStorage/issues/46
# WF 2021-08-23
SELECT ?substance ?substanceLabel ?formula ?structure ?CAS
WHERE { 
?substance wdt:P31 wd:Q11173.
?substance wdt:P231 ?CAS.
?substance wdt:P274 ?formula.
?substance wdt:P117  ?structure.
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 15

```
[try it!](https://query.wikidata.org/#%23%20List%20of%2015%20random%20chemical%20components%20with%20CAS-Number%2C%20formula%20and%20structure%0A%23%20see%20also%20https%3A//github.com/WolfgangFahl/pyLoDStorage/issues/46%0A%23%20WF%202021-08-23%0ASELECT%20%3Fsubstance%20%3FsubstanceLabel%20%3Fformula%20%3Fstructure%20%3FCAS%0AWHERE%20%7B%20%0A%3Fsubstance%20wdt%3AP31%20wd%3AQ11173.%0A%3Fsubstance%20wdt%3AP231%20%3FCAS.%0A%3Fsubstance%20wdt%3AP274%20%3Fformula.%0A%3Fsubstance%20wdt%3AP117%20%20%3Fstructure.%0ASERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22%5BAUTO_LANGUAGE%5D%2Cen%22.%20%7D%0A%7D%0ALIMIT%2015%0A)
## result
| substance                              | substanceLabel                   | formula      | structure                                                                                    | CAS        |
|----------------------------------------|----------------------------------|--------------|----------------------------------------------------------------------------------------------|------------|
| http://www.wikidata.org/entity/Q203174 | thalidomide                      | C₁₃H₁₀N₂O₄   | http://commons.wikimedia.org/wiki/Special:FilePath/Thalidomide-2D-skeletal-wavy.svg          | 50-35-1    |
| http://www.wikidata.org/entity/Q204030 | (E)-cinnamyl alcohol             | C₉H₁₀O       | http://commons.wikimedia.org/wiki/Special:FilePath/Cinnamyl%20alcohol.svg                    | 4407-36-7  |
| http://www.wikidata.org/entity/Q204594 | zinc protoporphyrin              | C₃₄H₃₂N₄O₄Zn | http://commons.wikimedia.org/wiki/Special:FilePath/Zinc%20protoporphyrin.svg                 | 15442-64-5 |
| http://www.wikidata.org/entity/Q204717 | zinc dithionite                  | O₄S₂Zn       | http://commons.wikimedia.org/wiki/Special:FilePath/Zinc%20dithionite.svg                     | 7779-86-4  |
| http://www.wikidata.org/entity/Q205562 | ziram                            | C₆H₁₂N₂S₄Zn  | http://commons.wikimedia.org/wiki/Special:FilePath/Ziram.png                                 | 137-30-4   |
| http://www.wikidata.org/entity/Q207843 | nitric oxide                     | NO           | http://commons.wikimedia.org/wiki/Special:FilePath/Nitric%20oxide.svg                        | 10102-43-9 |
| http://www.wikidata.org/entity/Q209281 | 2,7-di-tert-butylthiepin         | C₁₄H₂₂S      | http://commons.wikimedia.org/wiki/Special:FilePath/Ditertbutylthiepin.png                    | 83670-21-7 |
| http://www.wikidata.org/entity/Q209284 | 2,6-lutidine                     | C₇H₉N        | http://commons.wikimedia.org/wiki/Special:FilePath/Lutidine.png                              | 108-48-5   |
| http://www.wikidata.org/entity/Q209355 | ethephon                         | C₂H₆ClO₃P    | http://commons.wikimedia.org/wiki/Special:FilePath/Ethephon.png                              | 16672-87-0 |
| http://www.wikidata.org/entity/Q209407 | 2-hydroxy-5-octanoylbenzoic acid | C₁₅H₂₀O₄     | http://commons.wikimedia.org/wiki/Special:FilePath/Lipo-Hydroxy-S%C3%A4ure.svg               | 78418-01-6 |
| http://www.wikidata.org/entity/Q209415 | isopropyl iodide                 | C₃H₇I        | http://commons.wikimedia.org/wiki/Special:FilePath/Isopropyl%20iodide.svg                    | 75-30-9    |
| http://www.wikidata.org/entity/Q209444 | 2-methyltetrahydrofuran          | C₅H₁₀O       | http://commons.wikimedia.org/wiki/Special:FilePath/Methyltetrahydrofuran.svg                 | 96-47-9    |
| http://www.wikidata.org/entity/Q209444 | 2-methyltetrahydrofuran          | C₅H₁₀O       | http://commons.wikimedia.org/wiki/Special:FilePath/Methyltetrahydrofuran2.svg                | 96-47-9    |
| http://www.wikidata.org/entity/Q211433 | cyclohexane                      | C₆H₁₂        | http://commons.wikimedia.org/wiki/Special:FilePath/Skeletal%20formula%20of%20cyclohexane.svg | 110-82-7   |
| http://www.wikidata.org/entity/Q211737 | sodium sulfate                   | Na₂O₄S       | http://commons.wikimedia.org/wiki/Special:FilePath/Sodium%20sulfate.svg                      | 7757-82-6  |